# Задание 1 

Напишите функцию, которая принимает на вход любой текст в виде строки и применяет к нему метод n-грамм, n передаётся в параметрах функции и является целым числом. На выходе должен быть список с n-граммами.

In [10]:
# Функция, которая принимает на вход любой текст в виде строки и применяет к нему метод n-грамм

from nltk.util import ngrams, bigrams
import re 

def ft_ngrams(str, n):
  if str.find(' ') != -1:
    str = re.findall(r'(?i)[а-я]+', str) # Разобьем строку на слова с помощью re.findall
    return(list(ngrams(str, n)))# Применим метод ngrams
  else:
    return []

print(ft_ngrams('Если программа не выдает ошибку, значит, библиотека установлена успешно.', 3))

[('Если', 'программа', 'не'), ('программа', 'не', 'выдает'), ('не', 'выдает', 'ошибку'), ('выдает', 'ошибку', 'значит'), ('ошибку', 'значит', 'библиотека'), ('значит', 'библиотека', 'установлена'), ('библиотека', 'установлена', 'успешно')]


In [ ]:
def set_ngrams(str, n):
  from nltk.util import ngrams, bigrams
  import re 
  str = re.findall(r'(?i)[а-я]+', str) # Разобьем строку на слова с помощью re.findall
  return(set(ngrams(str, n)))# Применим метод ngrams

print(set_ngrams('Если программа не выдает ошибку, значит, библиотека установлена успешно.', 3))

{('Если', 'программа', 'не'), ('значит', 'библиотека', 'установлена'), ('библиотека', 'установлена', 'успешно'), ('выдает', 'ошибку', 'значит'), ('ошибку', 'значит', 'библиотека'), ('не', 'выдает', 'ошибку'), ('программа', 'не', 'выдает')}


# Задание 2

Напишите функцию для вычисления BLEU-score. На вход принимается две строки: перевод, предложенный алгоритмом, и референсный перевод. При подсчете метрики учитывайте n-граммы с n ∈ [1,2,3].

Перевод, предложенный алгоритмом: "Кошка вышла из дома и села на крыльцо"

Референсный перевод (ground truth): "Кошка вышла из комнаты и села на ступеньки"

На выходе должен быть ответ в виде процентов, округлённый до целых.

In [ ]:
# Функция для вычисления BLEUscore "ручной" вариант через сравнение сетов.

def Blue3(reference, hypothesis):
  n = len(list(reference.split()))
  brevity_penalty = min(1, len(list(hypothesis.split()))/len(list(reference.split())))
  setr1 = set_ngrams(reference, 1)
  seth1 = set_ngrams(hypothesis, 1)

  presigion1 = len(setr1 & seth1) / n
  setr2 = set_ngrams(reference, 2)
  seth2 = set_ngrams(hypothesis, 2)
  presigion2 = len(setr2 & seth2) / (n - 1)
  setr3 = set_ngrams(reference, 3)
  seth3 = set_ngrams(hypothesis, 3)

  presigion3 = len(setr3 & seth3) / (n - 2)
  return brevity_penalty * (presigion1  * presigion2 * presigion3) ** (1/3)

print(Blue3(reference, hypothesis))

0.5227579585747102


# Задание 3

Вспомните методы векторизации из прошлых уроков и проведите аналогичную классификацию, но также применив n-граммы. Сравните результаты.

Проведите классификацию текстов с применением n-грамм:


1.   Проведите предобработку
2.   Примените n-граммы с значением n = 2. Выше брать не будем, иначе расчеты могут сильно увеличиться по времени
3.   Проведите векторизацию с помощью Word2Vec
4.   Обучите на полученных данных модель градиентного бустинга
5.   Сравните эффективность модели, обученой ранее без применения n-грамм, с полученными моделями по показателю precision



## Приведем данные в необходимый для работы вид

In [6]:
!pip install pymorphy2 nltk
import nltk
nltk.download("punkt")
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import pandas as pd
!pip install gensim
from google.colab import drive
import os

drive.mount("/content/drive/", force_remount=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Mounted at /content/drive/


In [ ]:
# Проведем предобработку данных

df_p = pd.read_csv(('/content/drive/MyDrive/Pyton/positive.csv'), header = None, names = ['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count'], sep = ';', index_col=False)
df_n = pd.read_csv(('/content/drive/MyDrive/Pyton/negative.csv'), header = None, names = ['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count'], sep = ';', index_col=False)

df = pd.concat([df_p, df_n], axis=0) # Соединим два датафрейма
df.index = list(range(226834)) # Сделаем сквозную нумерецию
df.positive[df.positive==-1]=0 # Заменим признак негативных высказываний на 0

df.text = df.text.str.lower() # Приведем весь текст к строчным буквам
df.text = df.text.str.replace(r"[^А-Яа-я]"," ") # Оставим в тексте только русские слова, удалив числа, знаки препинания, специальные символы и слова написанные латиницей
df.text = list(map(word_tokenize, df.text)) # Разобьем тексты на слова с помощью word_tokenize

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text)) # Проведем лемматизацию полученных слов

def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text)) # Удалим стоп-слова из наших данных

preprocessed_df = df[["text", "positive"]] # Оставим только текст и значение positive

df = df.drop(df[df.text == ''].index, axis = 0) # Удалим пустые твиты

preprocessed_df.to_excel('/content/drive/MyDrive/Pyton/preprocessed.xlsx', index=False) # Сохраним обработанные данные


In [7]:
# Предобработанные данные хранятся в файле preprocessed.xlsx и теперь при перезагрузке будем сразу испорльзовать его

df_p = pd.read_excel('/content/drive/MyDrive/Pyton/preprocessed.xlsx')
df_p

,text,positive
0,"['школотый', 'поверь', 'самый', 'общество', 'п...",1
1,"['всё', 'таки', 'немного', 'похожий', 'мальчик...",1
2,"['идиотка', 'испугаться']",1
3,"['угол', 'сидеть', 'погибать', 'голод', 'ещ', ...",1
4,"['значит', 'страшилка', 'блин', 'посмотреть', ...",1
...,...,...
226829,"['каждый', 'хотеть', 'исправлять']",0
226830,"['скучать', 'вправлять', 'мозг', 'всё', 'равно...",0
226831,"['школа', 'говно', 'это', 'идти']",0
226832,"['тауриэль', 'грусть', 'обнять']",0


In [8]:
# Напишем функцию, которая из строки, которая выглядят, как список строк делает строку
# "['word', 'word', ... 'word']" -> "word word word"

def ft_str(str1):
  return str1.replace("', '", " ")[2 : -2]

In [11]:
text_2_gramms = list(ft_ngrams(elem, 2) for elem in list(map(ft_str, list(df_p.text))))

In [12]:
word_from_2_gramm = list(list('_'.join(j) for j in i) for i in text_2_gramms)

In [13]:
df_p['gramm'] = word_from_2_gramm

In [15]:
from gensim.models import Word2Vec

In [ ]:
# Используем gensim.models.phrasesмодуль, который позволяет автоматически определять фразы длиннее одного слова, используя статистику словосочетаний.
# Используя фразы, можно создать модель word2vec, где «слова» на самом деле представляют собой многословные выражения, такие как new_york_times

from gensim.models import Word2Vec
from gensim.models import Phrases

bigram_transformer = Phrases(word_from_2_gramm) # Обучим детектор биграмм

w2v = Word2Vec(bigram_transformer[word_from_2_gramm], min_count=1) # Применим обученный детектор к корпусу

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [17]:
w2v2 = Word2Vec(size=50, min_count=1) 
w2v2.build_vocab(df_p.gramm)
w2v2.train(df_p.gramm, total_examples=w2v2.corpus_count, epochs=10)

(12615590, 12615590)

In [20]:
w2v2.wv.get_vector(word_from_2_gramm[1][1])

array([-0.00413261,  0.01028042,  0.00273439, -0.00789322, -0.00611603,
       -0.01185608, -0.01721266,  0.00502513, -0.00033006,  0.01789803,
       -0.01035487, -0.01678614,  0.00855789,  0.00906763,  0.00478457,
       -0.02443067, -0.02700089,  0.01390506,  0.00644793,  0.00762786,
       -0.00080804,  0.00670501, -0.02096955, -0.01987435,  0.012967  ,
       -0.02114706, -0.01740248, -0.0032913 ,  0.00528294, -0.02020793,
        0.00461137,  0.01016803,  0.00242924,  0.01326174,  0.01903724,
       -0.0057761 , -0.00413389, -0.00279777,  0.00125301, -0.0236604 ,
       -0.01311484, -0.00691408, -0.0034867 ,  0.00790981,  0.00138631,
        0.00167093, -0.00105271,  0.02357635,  0.02987436, -0.01530246],
      dtype=float32)

In [22]:
import numpy as np

In [24]:
df_p['vector'] = list(np.mean(np.array(list(w2v2.wv.get_vector(j) for j in i)), axis = 0) for i in df_p.gramm)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [25]:
from sklearn.model_selection import train_test_split
!pip install xgboost

In [26]:
# Разобьем набор текстов на тренировочную и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(df_p.vector, df_p.positive, test_size=0.2, random_state=21)

In [28]:
X_train

114468    [-0.0014178, 0.00064833276, -0.0030441196, -0....
115015    [0.004725149, 0.0016996386, 0.0028317787, 0.00...
147222    [0.0018055621, 0.0021858667, -0.00021114372, -...
119060    [-0.0036846779, 0.0038996239, -0.0028481625, -...
74753     [-0.00576382, -0.0021149558, 0.0013430191, -0....
                                ...                        
81968     [-0.0019464877, 0.001380681, 0.004837542, -0.0...
140036    [-0.0033055625, 0.0012560157, 0.00046255664, -...
202552    [0.0030125116, -0.0035103569, -0.0008713889, -...
70863     [0.0034997219, 0.0021405788, -0.0011958218, -0...
80841     [0.0015324403, -0.0010657313, 0.00036654848, 0...
Name: vector, Length: 181467, dtype: object

In [34]:
X_train.shape

(181467,)

In [32]:
X.shape

(226834, 94864)

In [ ]:
# Применим модель xgboost. Основные гиперпараметры: max_depth - максимальная глубина деревьев модели и n_estimators - количество деревьев.
# min_count=2, epochs=20

from xgboost import XGBClassifier
from sklearn.metrics import classification_report

xgb = XGBClassifier(max_depth=10, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

---

---

---


---

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2


morph = pymorphy2.MorphAnalyzer()


def text_preprocessing(text):
    """Функция принимает строку и возвращает список слов в начальной форме"""
    text = text.lower()                                                         # приводим текст к нижнему регистру
    text = re.sub(r"[^А-Яа-я]", " ", text)                                      # удаляем все некириллические символы
    words = word_tokenize(text)                                                 # разбиваем тексты на списки слов
    words = [morph.parse(word)[0].normal_form for word in words]                # приводим слова к начальной форме
    words = [word for word in words if word not in stopwords.words("russian")]  # удаляем слова из стоп-листа
    return words

In [ ]:
text = 'Средства у нас есть, у нас ума не хватает'
prep_text = text_preprocessing(text)
print(prep_text)
print(type(prep_text))

['средство', 'ум', 'хватать']
<class 'list'>


In [ ]:
def text_to_2_gramms(text):
  text = ft_str(text)
  return ft_ngrams(text, 2)

In [ ]:
grammed_text = text_to_2_gramms(' '.join(prep_text))
grammed_text

[('едство', 'ум'), ('ум', 'хвата')]

In [ ]:
vectorized_grammed_text = word2vec(grammed_text)

In [ ]:
# проверка задания 3

text = 'Средства у нас есть, у нас ума не хватает'
prep_text = text_preprocessing(text)  # содержимое этой переменной сравнить с чек-листом
grammed_text = text_to_2_gramms(prep_text) # содержимое этой переменной сравнить с чек-листом

vectorized_grammed_text = word2vec(grammed_text)  # содержимое этой переменной сравнить с чек-листом -> набор чисел (вектор)
model_score = grad_boost(vectorized_grammed_text) # содержимое этой переменной сравнить с чек-листом -> precision, число от 0 до 1

# 2. Метод LDA (Latent Dirichlet Allocation - Латентное размещение Дирихле) и тематическое моделирование

LDA - один из методов тематического моделирования, позволяющий решить задачу классификации или кластеризации, где каждый класс или кластер содержит в себе тексты со схожими темами.

По сути тематическое моделирование - это такая мягкая кластеризация на текстах (хотя этот метод может быть расширен на данные любой природы, не только текстовые). Тематическая модель позволяет отнести текст к одной или нескольким темам с некторой вероятностью. Каждая тема, в свою очередь, характеризуются наиболее вероятными для неё словами.

К достоинствам можно отнести то, что это обучение без учителя, т.е. для обучения нам не требуются размеченные данные. 

Это же является и недостатком - тематическую модель довольно сложно настраивать, ведь нет чёткого представления чего мы хотим.

Для того, чтобы применять к датасету текстов LDA, необходимо преобразовать корпус в term-document matrix (Терм-документная матрица).

Терм-документная матрица — это матрица которая имеер размер $N \times W$, где
N — количество документов в корпусе, а W — размер словаря корпуса т.е. количество уникальных слов, которые встречаются в нашем корпусе. В i-й строке, j-м столбце матрицы находится число — сколько раз в i-м тексте встретилось j-е слово.

# Задание 4

Используя модель LDA, кластеризуйте твиты на положительные и негативные. Для этого:
1. Проведите предобработку данных
2. Представьте корпус твитов в виде Терм-документной матрицы (CountVectorizer), не забудьте о стоп-словах
3. Поэксперементируйте с параметром `max-iter` модели, подберите лучший вариант. [Документация](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)
4. В качестве оценки используйте степень правдоподобия (Log-likehood). Сделайте сравнительный анализ степеней правдоподобия, соответсвующих каждому рассмотренному набору параметров (например, в виде графиков зависимости степени правдоподобия от конкретного параметра).

In [2]:
import pandas as pd

In [3]:
# Предобработанные данные хранятся в файле preprocessed.xlsx и теперь при перезагрузке будем сразу испорльзовать его

df_p = pd.read_excel('/content/drive/MyDrive/Pyton/preprocessed.xlsx')
df_p

,text,positive
0,"['школотый', 'поверь', 'самый', 'общество', 'п...",1
1,"['всё', 'таки', 'немного', 'похожий', 'мальчик...",1
2,"['идиотка', 'испугаться']",1
3,"['угол', 'сидеть', 'погибать', 'голод', 'ещ', ...",1
4,"['значит', 'страшилка', 'блин', 'посмотреть', ...",1
...,...,...
226829,"['каждый', 'хотеть', 'исправлять']",0
226830,"['скучать', 'вправлять', 'мозг', 'всё', 'равно...",0
226831,"['школа', 'говно', 'это', 'идти']",0
226832,"['тауриэль', 'грусть', 'обнять']",0


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2


morph = pymorphy2.MorphAnalyzer()
corpus = df_p.text

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

array(['аа', 'ааа', 'аааа', ..., 'ёрса', 'ёрш', 'ёршик'], dtype=object)

In [ ]:
import pandas as pd

# Терм-документная матрица
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [5]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=2, max_iter=10)
lda.fit(X)

LatentDirichletAllocation(n_components=2)

In [18]:
lda.score(X)

-12801002.056882793

In [19]:
lda2 = LatentDirichletAllocation(n_components=2, max_iter=2)
lda2.fit(X)
lda2.score(X)

-12872769.791511888

In [23]:
lda20 = LatentDirichletAllocation(n_components=2, max_iter=20)
lda20.fit(X)
lda20.score(X)

-12759794.873599421